<a href="https://colab.research.google.com/github/Joracosu/03MAIR---Algoritmos-de-Optimizacion---2019/blob/master/AG1/Jos%C3%A9_Ram%C3%B3n_Conde_AG1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Actividad Guiada 1 de Algoritmos de Optimización

José Ramón Conde Suárez

https://colab.research.google.com/drive/1b0od-aP07ucKhmEqv2oQXfaofBG7L6km

## Ejercicio de las torres de Hanói

In [0]:
def torres (N, desde, hasta):
  if N==1:
    print('Lleva la ficha desde', str(desde),'hasta',str(hasta))
  else:
    torres(N-1, desde, 6-desde-hasta)
    print('Lleva la ficha desde', str(desde), 'hasta', str(hasta))
    torres(N-1, 6-desde-hasta, hasta)

# torres(3, 1, 3)

## Ejercicio de las n reinas

In [0]:
def reinas(N, solucion, etapa, j=0):
  for i in range(1, N + 1):
    solucion[etapa] = i
    if es_prometedora(solucion, etapa):
      if etapa == N - 1:
        j = j + 1
        print('Una solución es [',j,']:', solucion)
      else:
        j = reinas(N, solucion, etapa + 1, j)
    solucion[etapa] = 0
  return j

def es_prometedora(SOLUCION, etapa):
  for i in range(etapa + 1):
    if SOLUCION.count(SOLUCION[i]) > 1: return False
    for j in range(i + 1, etapa + 1):
      if abs(i-j) == abs(SOLUCION[i] - SOLUCION[j]): return False
  return True

# print('Total',reinas(8,([0,0,0,0,0,0,0,0]),0),'soluciones')


## Ejercicio de vecinos más cercanos


### Fuerza bruta
#### Todas las dimensiones

In [0]:
def puntos_mas_cercanos_bruto_multiD(lista):
  # se calcula un valor que seguro que es mayor que la distancia mínima de la lista
        # ORDEN DE COMPLEJIDAD:
        # k*(2*n+3)+4 = 2kn + 3k + 4 ~> O(n)
  auxiliar = 0
  for n in range(len(lista[0])-1):
    auxiliar += (max([k[n] for k in lista])-min([k[n] for k in lista]))**2
  minimo = (((auxiliar)**0.5)+1)*2
        #-------------------------------

        # ORDEN DE COMPLEJIDAD:
        # (6+5k)*(n**2-n) ~> O(n**2)
  for i in range(len(lista)-1): #para todos los valores de la lista menos el último
    for j in range(i+1,len(lista)): # comparar con el resto de valores de la lista
      p, q = lista[i], lista[j]

      # cálculo de la distancia euquidea independientemente de las dimensiones
      auxiliar = 0
      for n in range(len(lista[0])-1):
        auxiliar += (p[n]-q[n])**2
      dist = (auxiliar)**0.5
      
      if dist < minimo:
        # se reinicia la lista
        minimo = dist
        pareja = [[p, q]]
      elif dist == minimo:
        # se añade el punto a la lista
        pareja += [[p, q]]
        #-------------------------------

        # ORDEN DE COMPLEJIDAD:
        # O(n**2) + O(n**2) + O(n) ~> O(n**2)
  k = sorted(pareja)
  k = [k[i] for i in range(len(k)) if i == 0 or k[i] != k[i-1]]
  return minimo, k

### Divide y venceras
#### Todas las dimensiones

In [0]:
import random
import numpy as np
from time import time
def puntos_mas_cercanos_divide_multiD(lista, dimension = -1):

  if dimension == -1: dimension = len(lista[0]) - 1

  if len(lista) <= 3:
    # para aplicar el método, lo primero es ordenar la lista
    lista = sorted(lista)   
    # cuando la división resulta en una lista de tamaño 3 o menor
    min_0, pareja_0 = puntos_mas_cercanos_bruto_multiD(lista) 
    
  else:
    # para aplicar el método, se ordena primero por la dimensión última
    if dimension == 1:
      lista = sorted(lista)   
    else:
      lista = sorted(lista, key=lambda punto : punto[dimension - 1]) 
    # se divide la lista en dos partes
    largo = len(lista)      
    lista_1 = lista[:int(largo/2)]  
    lista_2 = lista[int(largo - largo/2):]  
    
    # se aplica recurrentemente el algoritmo a cada división
    min_1, pareja_1 = puntos_mas_cercanos_divide_multiD(lista_1, dimension)  
    min_2, pareja_2 = puntos_mas_cercanos_divide_multiD(lista_2, dimension)  

    # Estudio de la frontera:
    #    Al tratarse de tres dimensiones, hay que estudiar TODOS los puntos que se
    #    encuentran a una distancia en z menor que la distancia mínima encontrada
    #    entre las dos listas. Para ello se crea una lista que contenga dichos 
    #    puntos independientemente de su X y de su Y y se calcula su distancia mínima por 
    #    divide y venceras en 2D
    if dimension == 1:
      p, q = lista_1[len(lista_1)-1], lista_2[0]
      min_3, pareja_3 = puntos_mas_cercanos_bruto_multiD([p, q])
    else:
      p_medio = lista[int(largo/2)-1]   
      dm = min(min_1, min_2)    
      lista = np.array(lista)   
      # print(dimension)
      lista_3 = [tuple(p) for p in list(lista[[abs(i[dimension - 1] - p_medio[dimension - 1]) <= dm for i in lista]])] 

      if len(lista_3)>1: min_3, pareja_3 = puntos_mas_cercanos_divide_multiD(lista_3, dimension - 1)    
      else: min_3 = max(min_1, min_2) + 1  # con esto se declara un valor para min_3 pero se decarta    

    # se almacenan todas las parejas de puntos de la distancia mínima
    # min_0 = min(min_1, min_2, min_3)
    if min_1 == min_2:
      if min_1 == min_3:
        min_0, pareja_0 = min_1, pareja_1 + pareja_3 + pareja_2
      elif min_1 < min_3:
        min_0, pareja_0 = min_1, pareja_1 + pareja_2
      else:
        min_0, pareja_0 = min_3, pareja_3
    elif min_1 > min_2:
      if min_2 == min_3:
        min_0, pareja_0 = min_2, pareja_3 + pareja_2
      elif min_2 < min_3:
        min_0, pareja_0 = min_2, pareja_2
      else:
        min_0, pareja_0 = min_3, pareja_3
    else:
      if min_1 == min_3:
        min_0, pareja_0 = min_1, pareja_1 + pareja_3
      elif min_1 < min_3:
        min_0, pareja_0 = min_1, pareja_1
      else:
        min_0, pareja_0 = min_3, pareja_3

  # finalmente se eliminan puntos duplicados
  k = sorted(pareja_0)
  k = [k[i] for i in range(len(k)) if i == 0 or k[i] != k[i-1]]
  return min_0, k

### Aplicación a listas

In [0]:
import random
from time import time

def comparativa_metodos(dimension, precision, rango):
  print('\nDimensión',dimension)
  lista_de_numeros = []
  for x in range(rango):
    nuevo_punto = []
    for i in range(dimension):
      nuevo_punto += [random.randrange(0,precision)]
    nuevo_punto += [[x+1]]
    lista_de_numeros += [tuple(nuevo_punto)]  
  lista = sorted(lista_de_numeros)    

  metodos = {'Fuerza bruta' : puntos_mas_cercanos_bruto_multiD,
             'Divide y vencerás' : puntos_mas_cercanos_divide_multiD}

  for nombre, metodo in metodos.items():
    print(nombre)
    tiempo_1 = time()
    min_a, puntos_a = metodo(lista)
    tiempo_2 = time()
    print(min_a, puntos_a)
    print('Segundos empleados:', tiempo_2 - tiempo_1)
    print('')
  print('---------------------------------------------------')

comparativa_metodos(1, 100000000, 3000)
comparativa_metodos(2, 20000, 2500)
comparativa_metodos(3, 2000, 2000)
comparativa_metodos(4, 2000, 1700)
comparativa_metodos(5, 2000, 1400)
# a partir de la sexta dimensión, el algoritmo de Divide y Vencerás se vuelve más lento que el de fuerza bruta
comparativa_metodos(6, 2000, 1200)




Dimensión 1
Fuerza bruta
10.0 [[(85155432, [2265]), (85155442, [2182])]]
Segundos empleados: 4.373668670654297

Divide y vencerás
10.0 [[(85155432, [2265]), (85155442, [2182])]]
Segundos empleados: 0.01706671714782715

---------------------------------------------------

Dimensión 2
Fuerza bruta
5.830951894845301 [[(18856, 4340, [1443]), (18861, 4337, [1214])]]
Segundos empleados: 4.053486585617065

Divide y vencerás
5.830951894845301 [[(18856, 4340, [1443]), (18861, 4337, [1214])]]
Segundos empleados: 0.11850643157958984

---------------------------------------------------

Dimensión 3
Fuerza bruta
13.076696830622021 [[(966, 1729, 693, [1510]), (977, 1734, 698, [303])]]
Segundos empleados: 3.3673791885375977

Divide y vencerás
13.076696830622021 [[(966, 1729, 693, [1510]), (977, 1734, 698, [303])]]
Segundos empleados: 0.34186697006225586

---------------------------------------------------

Dimensión 4
Fuerza bruta
41.701318923986086 [[(565, 575, 261, 1454, [296]), (589, 570, 268, 14